In [1]:
import tensorflow as tf 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten


from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
from skimage.io import imsave

import os
import numpy as np
from keras.models import load_model

2025-01-16 23:26:30.342786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-16 23:26:31.075242: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-01-16 23:26:31.075319: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2025-01-16 23:26:31.075326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import phase2 as VP

In [3]:
# Dataset parameters 
dataset_path = '../../../data/processed/smallerSampleDataset/'
reshape_size = [224,224,3]
phishing_test_size = 0.4
# num_targets = 155 
num_targets = 5

# Model parameters
input_shape = [224,224,3]
margin = 2.2
new_conv_params = [5,5,512]

# Training parameters
start_lr = 0.00002
output_dir = '../../../notebooks/'
saved_model_name = 'model' #from first training 
new_saved_model_name = 'model2'
save_interval = 2000
batch_size = 32 
n_iter = 50000
lr_interval = 250
# hard examples training 
num_sets = 100
iter_per_set = 8
n_iter = 30
batch_size = 32

# Load dataset:
- Load training screenshots per website
- Load Phishing screenshots per website

In [ ]:
### read_imgs_per_website


In [4]:
# Read images legit (train)
data_path = dataset_path + 'trusted_list/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
# imgs_num = 9363
imgs_num = 420

all_imgs_train,all_labels_train,all_file_names_train = VP.read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
# imgs_num = 1195
imgs_num = 160

all_imgs_test,all_labels_test,all_file_names_test = VP.read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

# Load the same train/split in phase 1
phish_test_idx = np.load(output_dir+'test_idx.npy')
phish_train_idx = np.load(output_dir+'train_idx.npy')

X_test_phish = all_imgs_test[phish_test_idx,:]
y_test_phish = all_labels_test[phish_test_idx,:]

X_train_phish = all_imgs_test[phish_train_idx,:]
y_train_phish = all_labels_test[phish_train_idx,:]

../../../data/processed/smallerSampleDataset/trusted_list/absa
../../../data/processed/smallerSampleDataset/trusted_list/alibaba
../../../data/processed/smallerSampleDataset/trusted_list/amazon
../../../data/processed/smallerSampleDataset/trusted_list/apple
../../../data/processed/smallerSampleDataset/trusted_list/boa
../../../data/processed/smallerSampleDataset/phishing/absa
../../../data/processed/smallerSampleDataset/phishing/alibaba
../../../data/processed/smallerSampleDataset/phishing/amazon
../../../data/processed/smallerSampleDataset/phishing/apple
../../../data/processed/smallerSampleDataset/phishing/boa


# Order and label targets

In [5]:
labels_start_end_train_legit = VP.all_targets_start_end(num_targets,y_train_legit)

# Finding Hard subsets

In [ ]:
###

In [6]:
# Don't sample negative image from the same category as the positive image (e.g. google and google drive)
# Create clusters of same-company websites (e.g. all microsoft websites)


targets_file = open(data_path+'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()


        
#targets names of parent and sub websites
target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]



parents_ids,sub_target_lists_idx  = VP.get_associated_targets_idx(target_lists,all_targets)




# Triplet Sampling

# Load Model

In [7]:



full_model = load_model(output_dir+saved_model_name+'.h5', custom_objects={'loss': VP.loss})




from keras import optimizers
optimizer = optimizers.Adam(lr = start_lr)
full_model.compile(loss=VP.custom_loss(margin),optimizer=optimizer)

2025-01-16 23:30:50.215650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-16 23:30:50.231693: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-16 23:30:50.231734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-16 23:30:50.232253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

# Training

In [10]:
def find_fixed_set_idx(labels_start_end_train_legit2, num_target):
    website_random_idx = np.zeros([num_target,])
    for i in range(0,num_target):
        class_idx_start_end = labels_start_end_train_legit2[i,:]
        website_random_idx[i] = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    return website_random_idx

In [12]:
n=1 #number of wrong points 

#all training images
X_train = np.concatenate([X_train_legit,X_train_phish])
y_train = np.concatenate([y_train_legit,y_train_phish])

#subset training
X_train_new = np.zeros([num_targets*2*n,X_train_legit.shape[1],X_train_legit.shape[2],X_train_legit.shape[3]])
y_train_new = np.zeros([num_targets*2*n,1])

targets_train = np.zeros([batch_size,1])
tot_count = 0 

print("Starting training process!")
print("\n ------------- \n")
for k in range(0,num_sets):
    print("Starting a new set!")
    print("\n ------------- \n")
    X_train_legit = all_imgs_train
    y_train_legit = all_labels_train
    
    fixed_set_idx = find_fixed_set_idx(labels_start_end_train_legit, num_targets)
    fixed_set = X_train_legit[fixed_set_idx.astype(int),:,:,:]
    
    for j in range(0,iter_per_set):
        model = full_model.layers[3]
        X_train_new,y_train_new,labels_start_end_train = VP.find_main_train(model,fixed_set,num_targets)
        for i in range(1, n_iter):
            tot_count = tot_count + 1
            inputs=VP.get_batch_for_phase2(batch_size, fixed_set, num_targets)
            loss_iteration=full_model.train_on_batch(inputs,targets_train)
            
            print("\n ------------- \n")
            print('Iteration: '+ str(i) +'. '+ "Loss: {0}".format(loss_iteration))

            if tot_count % save_interval == 0:
                VP.save_model(full_model)

            if tot_count % lr_interval ==0:
                start_lr = 0.99*start_lr
                K.set_value(full_model.optimizer.lr, start_lr)

VP.save_model(full_model)

Starting training process!

 ------------- 

Starting a new set!

 ------------- 



NameError: name 'X_train_legit' is not defined

# Calculate Embeddings

In [ ]:
shared_model = full_model.layers[3]

whitelist_emb = shared_model.predict(X_train_legit,batch_size=64)
np.save(output_dir+'whitelist_emb2',whitelist_emb)
np.save(output_dir+'whitelist_labels2',y_train_legit )

phishing_emb = shared_model.predict(all_imgs_test,batch_size=64)
np.save(output_dir+'phishing_emb2',phishing_emb)
np.save(output_dir+'phishing_labels2',all_labels_test )